In [1]:
import json
from tqdm import tqdm
import pandas as pd
from utils.preprocessing import *

In [8]:
start_index = 50000
end_index = 75000

In [3]:
# get running data from endomondo
input_file = "archive/endomondoHR_proper.json"
output_file = "archive/endomondoHR_fixed.json"

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        corrected_line = line.replace("'", '"')  # Replace single quotes with double quotes
        outfile.write(corrected_line)


data = []
with open(output_file, "r", encoding="utf-8") as f:
    for line in tqdm(f):
        data.append(json.loads(line))  # Load each line separately

print(f"Loaded {len(data)} JSON objects")


167783it [00:43, 3875.30it/s]

Loaded 167783 JSON objects


In [4]:
# limit to only running data and specified index range
tqdm.pandas()
df = pd.DataFrame(data)
run_routes = df[df['sport']=='run']

run_routes.reset_index(inplace=True, drop=True)

run_routes['distance'] = run_routes.progress_apply(lambda row: get_route_distance(row['latitude'], row['longitude']), axis=1)
run_routes['duration'] = run_routes.apply(lambda row: calculate_route_duration(row['timestamp']), axis=1)
run_routes['start_end_dist'] = run_routes.apply(lambda row: get_start_end_dist(row['latitude'], row['longitude']), axis=1)

# ensure indexes are in bounds
start_index = max(0, start_index)
end_index = min(end_index, len(run_routes))

run_routes = run_routes.iloc[start_index:end_index, :]
run_routes.head()

100%|██████████| 70591/70591 [00:15<00:00, 4544.13it/s]
/var/folders/d4/c154mt9s11x5q965j91l06qw0000gn/T/ipykernel_52261/4208802304.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_routes['distance'] = run_routes.progress_apply(lambda row: get_route_distance(row['latitude'], row['longitude']), axis=1)
/var/folders/d4/c154mt9s11x5q965j91l06qw0000gn/T/ipykernel_52261/4208802304.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  run_routes['duration'] = run_routes.apply(lambda row: calculate_route_dura

,longitude,altitude,latitude,sport,id,heart_rate,gender,timestamp,url,userId,speed,distance,duration,start_end_dist
50000,"[21.833996018394828, 21.833990653976798, 21.83...","[72.2, 72.2, 75.0, 79.6, 85.0, 86.2, 87.2, 87....","[51.58289453946054, 51.58289462327957, 51.5828...",run,278936964,"[72, 72, 76, 76, 76, 93, 107, 107, 115, 123, 1...",male,"[1387809859, 1387809860, 1387809866, 138780987...",https://www.endomondo.com/users/5746423/workou...,5746423,NaN,9.317126,90.850000,0.012225
50001,"[21.833832319825888, 21.8338143825531, 21.8336...","[96.6, 96.4, 94.8, 88.6, 87.6, 91.0, 98.4, 102...","[51.58287308178842, 51.58286126330495, 51.5827...",run,278797765,"[72, 72, 86, 93, 105, 112, 127, 136, 140, 141,...",male,"[1387739310, 1387739311, 1387739316, 138773933...",https://www.endomondo.com/users/5746423/workou...,5746423,NaN,15.735276,122.733333,0.019923
50002,"[21.834051590412855, 21.83392812497914, 21.833...","[96.0, 97.2, 99.8, 103.6, 108.0, 114.2, 122.0,...","[51.58304206095636, 51.58314616419375, 51.5832...",run,278163943,"[63, 68, 78, 86, 102, 104, 112, 124, 129, 129,...",male,"[1387488342, 1387488347, 1387488351, 138748835...",https://www.endomondo.com/users/5746423/workou...,5746423,NaN,10.133904,94.750000,0.005726
50003,"[21.83403122238815, 21.83400247246027, 21.8336...","[88.2, 88.2, 88.8, 89.6, 90.2, 91.0, 93.4, 93....","[51.58312671817839, 51.58311640843749, 51.5828...",run,277676087,"[91, 94, 104, 118, 118, 124, 123, 142, 149, 14...",male,"[1387314478, 1387314480, 1387314492, 138731449...",https://www.endomondo.com/users/5746423/workou...,5746423,NaN,9.925788,89.833333,0.003217
50004,"[21.8337263725698, 21.8337332457304, 21.833733...","[2.4, 2.4, 3.6, 11.2, 16.4, 35.0, 45.0, 48.6, ...","[51.583170806989074, 51.583179691806436, 51.58...",run,277436339,"[75, 75, 86, 104, 121, 124, 129, 135, 138, 138...",male,"[1387226671, 1387226673, 1387226678, 138722669...",https://www.endomondo.com/users/5746423/workou...,5746423,NaN,10.634698,99.433333,0.011967


In [7]:
run_routes.to_csv('run_routes_unprocessed.csv')

In [4]:
import json
from tqdm import tqdm
import pandas as pd
from utils.preprocessing import *

run_routes = pd.read_csv('run_routes_unprocessed.csv')


In [5]:
run_routes['longitude'] = run_routes['longitude'].apply(lambda x: json.loads(x))
run_routes['latitude'] = run_routes['latitude'].apply(lambda x: json.loads(x))
run_routes.set_index('Unnamed: 0', inplace=True)

In [6]:

processed_routes = parallel_process(run_routes)


  2%|▏         | 467/20591 [04:48<3:22:05,  1.66it/s]

  2%|▏         | 479/20591 [04:56<3:23:59,  1.64it/s]

  2%|▏         | 506/20591 [05:12<2:58:34,  1.87it/s]

  2%|▏         | 508/20591 [05:14<3:26:52,  1.62it/s]

  2%|▏         | 513/20591 [05:17<4:19:34,  1.29it/s]

...

 98%|█████████▊| 20230/20591 [3:31:04<04:06,  1.47it/s]

 98%|█████████▊| 20260/20591 [3:31:24<03:01,  1.82it/s]

 99%|█████████▉| 20388/20591 [3:32:54<02:21,  1.43it/s]

 99%|█████████▉| 20455/20591 [3:33:35<01:08,  1.99it/s]

100%|██████████| 20591/20591 [3:35:04<00:00,  1.60it/s]


In [9]:

folder_name = "data"

# Check if the folder already exists
if not os.path.exists(folder_name):
    # Create the folder
    os.makedirs(folder_name)

processed_routes.to_csv(f'data/processed_routes_{start_index}_{end_index}.csv', index=False)
print(f"Processed routes saved to data/processed_routes_{start_index}_{end_index}.csv")

Processed routes saved to data/processed_routes_50000_75000.csv
